In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()
import plotly.graph_objects as go
fig = go.Figure()

from wordcloud import WordCloud

import plotly.express as px 
%matplotlib inline

In [ ]:
path = '/kaggle/input/movies-on-netflix-prime-video-hulu-and-disney/MoviesOnStreamingPlatforms_updated.csv'
data = pd.read_csv(path)
data.head()

# **Perfroming EDA**

In [ ]:
cols = data.columns.tolist()
cols

### **Dropping Columns - Id and Unnamed**

In [ ]:
data.drop(['Unnamed: 0','ID',], axis=1, inplace = True)
cols = data.columns.tolist()
cols

In [ ]:
data.info()

### **Checking Missing Data**

In [ ]:
print("Percentage Missing Data")
(data.isnull().sum()/data.shape[0])*100

### We see that alot of columns have null/missing data. We won't drop these columns as we are perfoming Analysis. We can drop the Rotten Tomatoes column as it has approx 69% missing data.

### Age column has values like 18+ 16+. We fill format the column to have only numerical data such as 18 and 16 which means that a movie is 18+ or 16+ respectively. We might need to have only Numerical data in the Age column in future

In [ ]:
data.Age.value_counts()

In [ ]:
age_map = {'18+' : 18, '7+' : 7, '13+': 13, 'all' : 0, '16+' : 16}
data['intAge'] = data['Age'].map(age_map)

In [ ]:
data.head()

In [ ]:
data['Rotten Tomatoes'].value_counts().sort_values(ascending=False)

### Process Rotten Tomatoes column and convert it to remove %. 

In [ ]:
data['New_Rotten_Tomatoes'] = data['Rotten Tomatoes'].str.replace("%","")
for i in data['New_Rotten_Tomatoes']:
    if i is str:
        i = i.astype(int)
    

In [ ]:
data.info()

In [ ]:
data['New_Rotten_Tomatoes']

### It is still showing as Object type. We will again process it if we encounter any issue while analyzing the data!

### *For now all the dats types looks fine. We can proceed with Plotting Graphs*

# **Plotting Graphs**

## **Checking Number of Movies in a given Age group per Streaming Service**

In [ ]:
data['Age'].value_counts().iplot('bar', xTitle='Age Group', 
                                        yTitle='Count of Movies', 
                                        title="Number of Movies in specific age group in All services")

<h3 style="color: blue;">Most movies are rated as 18+ and very less are fit for all age groups!</h3>

### Analyzing number of Movies in speific age group in NetFlix

In [ ]:
data_netflix = data.copy()
data_netflix = data_netflix[data['Netflix']==1]
data_netflix['Age'].value_counts().iplot('bar', colors='Blue', xTitle='Age Group', 
                                        yTitle='Count of Movies', 
                                        title="Number of Movies in specific age group in NetFlix")

<h3 style="color: blue;">Around 1/5 of 18+ movies, 1/4 13+ moveis, 1/5 7+ moveis and 1/4 16+ movies are from NetFlix </h3>

In [ ]:
data_hulu = data.copy()
data_hulu = data_hulu[data['Hulu']==1]
data_hulu['Age'].value_counts().iplot('bar', colors='Red', xTitle='Age Group', 
                                        yTitle='Count of Movies', 
                                        title="Number of Movies in specific age group in Hulu")

<h3 style="color: blue;">Around 1/11 of 18+ movies, 1/10 13+ moveis, 1/13 7+ moveis and 1/19 16+ movies are from Hulu </h3>

In [ ]:
data_prime = data.copy()
data_prime = data_prime[data['Prime Video']==1]
data_prime['Age'].value_counts().iplot('bar', colors='Black', xTitle='Age Group', 
                                        yTitle='Count of Movies', 
                                        title="Number of Movies in specific age group in Prime Video")

<h3 style="color: blue;">Around 1/11 of 18+ movies, 1/10 13+ moveis, 1/13 7+ moveis and 1/19 16+ movies are from Hulu </h3>

In [ ]:
data_disney = data.copy()
data_disney = data_disney[data['Disney+']==1]
data_disney['Age'].value_counts().iplot('bar', colors='Purple', 
                                        xTitle='Age Group', 
                                        yTitle='Count of Movies', 
                                        title="Number of Movies in specific age group in Disney+")

<h3 style="color: blue;">Most of the movies streamed on Disney+ is suitable for all age gorup. We can see that, per the dataset only 3 18+ movies are listed in the Disney+</h3>

## **Rotten Tomatoes Score**
### A Tomatometer score is calculated for a movie or TV show after it receives at least five reviews. When at least 60% of reviews for a movie or TV show are positive, a red tomato is displayed to indicate its Fresh status.
### Rotten Tomatoes gives films a score out of 100 based on the averaged reviews of professional film critics. If a film gets a rating of 60 or more it gets a 'fresh' red tomato on the site. Less than 60 and it gets a rotten tomato.
#### : Source - Google

In [ ]:
data['Rotten Tomatoes'].value_counts().iplot(kind = 'bar', colors = 'Cyan', xTitle = "Ratings", yTitle="Number of Movies", title="Overall Rotten Tomato Ratings")

In [ ]:
rotten_tomato_scores = pd.DataFrame({'Streaming Service': ["Prime Video", "Hulu","Disney+","NetFlix"],
                                    'Rotten Tomato Score' : [data_prime['Rotten Tomatoes'].value_counts()[0], 
                                                             data_hulu['Rotten Tomatoes'].value_counts()[0],
                                                             data_disney['Rotten Tomatoes'].value_counts()[0],
                                                             data_netflix['Rotten Tomatoes'].value_counts()[0]]})
rotten_tomato_scores.head()

In [ ]:
rotten_tomato_scores.sort_values(ascending=False, by="Rotten Tomato Score").iplot(kind='bar', x='Streaming Service', y='Rotten Tomato Score', 
                           color='Violet', xTitle="Streaming Service", 
                           yTitle="Count of Movies with Score of 100%", 
                           title="Streaming Service with 100% Rotten Tomato Score")

<h3 style="color: purple;">Prime is home to most movies with 100% Rotten Tomato Score</h3>

## **IMDB Ratings**

In [ ]:
data['IMDb'].value_counts().iplot(kind="bar", color="Red", xTitle='IMDb Ratings', yTitle="Count of Movies", title = "Count of Movies vs IMDb Ratings")

<h3 style="color: purple;">We see that approx 553 movies have IMDb rating as 6.5 where as 407 movies have received Rotten Tomato Rating as 100%!</h3>

In [ ]:
def get_imdb_count_per_service(d):
    # This function returns the number of movies in a service having IMDb score greater than 7.5
    num_of_movies = 0
    for key,value in d.items():
        if key>=7.5:
            num_of_movies+=value
    return num_of_movies

In [ ]:
imdb_prime_count = get_imdb_count_per_service(dict(data_prime['IMDb'].value_counts().sort_values(ascending=False)))
imdb_hulu_count = get_imdb_count_per_service(dict(data_hulu['IMDb'].value_counts().sort_values(ascending=False)))
imdb_disey_count = get_imdb_count_per_service(dict(data_disney['IMDb'].value_counts().sort_values(ascending=False)))
imdb_netflix_count = get_imdb_count_per_service(dict(data_netflix['IMDb'].value_counts().sort_values(ascending=False)))

imdb_scores = pd.DataFrame({'Streaming Service': ["Prime Video", "Hulu","Disney+","NetFlix"],
                                    'IMDb Score' :[imdb_prime_count, imdb_hulu_count, imdb_disey_count, imdb_netflix_count] })
imdb_scores.head()

In [ ]:
imdb_scores.sort_values(ascending=False, by='IMDb Score').iplot(kind="bar", color="Cyan",x='Streaming Service', 
                  xTitle='Streaming Service', 
                  yTitle="Count of Movies with IMDB Score >=7.5", 
                  title = "Streaming Services with Movies having IMDB >= 7.5")

<h3 style="color: purple;">Prime Video has highest number of movies with IMDB more than 7.5!</h3>

## **Explore Languages**

In [ ]:
data['Language'].value_counts()

### We see that the data represent - Movie and in which all languages it is available in. But we want is count of every language. As in Number of movies in English, in Spanish and etc.

#### Getting total languages in dataset

In [ ]:
%%time
languages_dict = dict(data['Language'].value_counts())
languages = set()
for lang,count in languages_dict.items():
    curr_lang = lang
    curr_langs = curr_lang.split(",")
    for i in curr_langs:
        if i in languages:
            continue
        else:
            languages.add(i.lower())
languages = list(languages)
print("Total number of languages are : ", len(languages))

#### Counting each language

In [ ]:
%%time
languages_count = dict()
for lang,count in languages_dict.items():
    curr_lang = lang.split(",")
    for i in curr_lang:
        if i in languages_count.keys():
            languages_count[i] = languages_count.get(i) + 1
        else:
            languages_count[i] = 1

#### We now have a dictionary having all the languages and their count

In [ ]:
lang_count_df = pd.DataFrame(languages_count.items(), columns=['Language', 'Count'])
lang_count_df.head()

## Plotting Graph for Language and Count

In [ ]:
lang_count_df.sort_values(ascending=False, by='Count')[:20].iplot(kind='bar', 
                                                                  x='Language', 
                                                                  xTitle='Language', 
                                                                  yTitle='Count', colors='Green', 
                                                                  title='Language vs Count')

Top 20 languages in Streaming Services

In [ ]:
lang_count_df_copy = lang_count_df[:20]
fig = px.pie(values=lang_count_df_copy['Count'], names=lang_count_df_copy['Language']) 
fig.show()

<h3 style="color: purple;">Most of the movies in these Streaming servies are in English.</h3>

## **Movie/Show Runtime**

In [ ]:
pd.DataFrame(dict(data['Runtime'].value_counts().sort_values(ascending=False)[:20]).items(), columns=['Runtime', 'Count']).iplot(kind='bar' ,
                                                                                                                                 x='Runtime', 
                                                                                                                                 xTitle='Runtime', 
                                                                                                                                 yTitle='Count', title='Runtime vs Count',
                                                                                                                                colors='Magenta')

<h3 style="color: blue;">Most of the movies have run time of approx 90 minutes.</h3>

## Directors with Most Movies

In [ ]:
len(data['Directors'])

In [ ]:
directors = list(set(data['Directors']))
directors.pop(0) #TO REMOVE NAN VALUE
len(directors)

### By converting it to Set we reduced the number from 16k to 11k

In [ ]:
new_director = set()
for d in directors:
    curr_d = d.split(",")
    for direc in curr_d:
        if direc in new_director:
            continue
        else:
            new_director.add(direc)
len(new_director)

### Most of the movies have more than 1 director, hence extarcted all the names of directors in the dataset

In [ ]:
%%time
new_data = data[data['Directors'].notna()]
directors_count = dict()
direc_in_data = list(new_data['Directors'])
for xdir in direc_in_data:
    curr_dirs = xdir.split(",")
    for xd in curr_dirs:
        if xd in directors_count.keys():
            directors_count[xd] = directors_count.get(xd) + 1
        else:
            directors_count[xd] = 1

In [ ]:
directors_count_df = pd.DataFrame(directors_count.items(), columns=['Director', 'Count'])
directors_count_df.sort_values(ascending=False, by='Count').head()

In [ ]:
directors_count_df.sort_values(ascending=False, by='Count')[:20].iplot(kind='bar', 
                                                                  x='Director', 
                                                                  xTitle='Director', 
                                                                  yTitle='Count', colors='Blue', 
                                                                  title='Director vs Count')

### Jay Chapman has directed most number of movies as per our calculations. Lets see the moveis he has directed

In [ ]:
data[data['Directors']=='Jay Chapman']

<h3 style="color:red;">Here the List of moveis and other details for movies directed by Jay Chapman.</h3>

## **Movies available in more than 1 platforms**

### **Movies available in Netflix and Prime Video**

In [ ]:
temp_data = data[data['Netflix']==1]
temp_data = temp_data[temp_data['Prime Video']==1]
temp_data

In [ ]:
#list(temp_data['Title'])

In [ ]:
plt.subplots(figsize = (10,10))

wordcloud = WordCloud (
                    background_color = 'white',
                    width = 720,
                    height = 720
                        ).generate(' '.join(temp_data['Title']))
plt.imshow(wordcloud) # image show
plt.axis('off') # to off the axis of x and y
plt.show()

### **Movies available in Netflix and Hulu**

In [ ]:
temp_data_nh = data[data['Netflix']==1]
temp_data_nh = temp_data_nh[temp_data_nh['Hulu']==1]
#list(temp_data_nh['Title'])

In [ ]:
plt.subplots(figsize = (10,10))

wordcloud = WordCloud (
                    background_color = 'black',
                    width = 720,
                    height = 720
                        ).generate(' '.join(temp_data_nh['Title']))
plt.imshow(wordcloud) # image show
plt.axis('off') # to off the axis of x and y
plt.show()

## **Exploring Genres**

In [ ]:
%%time
genres_unclean = dict(data['Genres'].value_counts())
genres = set()
for g,count in genres_unclean.items():
    curr_g = g.split(",")
    for xg in curr_g:
        if xg in genres:
            continue
        else:
            genres.add(xg)

In [ ]:
%%time
count_genres = dict()
for g,count in genres_unclean.items():
    curr_g = g.split(",")
    for xg in curr_g:
        if xg in count_genres.keys():
            count_genres[xg] = count_genres.get(xg)+1
        else:
            count_genres[xg] = 1

In [ ]:
count_genres_df = pd.DataFrame(count_genres.items(), columns=['Genre', 'Count'])

In [ ]:
count_genres_df.sort_values(ascending=False, by='Count').iplot(kind="bar", x='Genre', xTitle='Genre', yTitle='Count', title='Count of Genres', color='pink')

In [ ]:
plt.subplots(figsize = (10,10))

wordcloud_genre = WordCloud (
                    background_color = 'white',
                    width = 720,
                    height = 720
                        ).generate(' '.join(count_genres_df['Genre']))
plt.imshow(wordcloud_genre) # image show
plt.axis('off') # to off the axis of x and y
plt.show()

<h3 style="color:blue;">Most of the movies in the platforms belong to Drama genre</h3>

## **Top Movies in each streaming platforms**

#### In this top movies will be those, having IMDB > 8.5

### NetFlix

In [ ]:
data_netflix_top = data_netflix[data_netflix['IMDb']>8.5]
data_netflix_top = data_netflix_top[['Title', 'IMDb']].sort_values(ascending=False, by='IMDb')

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='IMDb',y='Title',data=data_netflix_top, palette='deep')
plt.title('Top NetFlix Movies')
plt.show()

### Hulu

In [ ]:
data_hulu_top = data_hulu[data_hulu['IMDb']>8.5]
data_hulu_top = data_hulu_top[['Title', 'IMDb']].sort_values(ascending=False, by='IMDb')

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='IMDb',y='Title',data=data_hulu_top, palette='husl')
plt.title('Top Hulu Movies')
plt.show()

### Disney+

In [ ]:
data_disney_top = data_disney[data_disney['IMDb']>8.5]
data_disney_top = data_disney_top[['Title', 'IMDb']].sort_values(ascending=False, by='IMDb')

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='IMDb',y='Title',data=data_disney_top, palette='husl')
plt.title('Top Disney+ Movies')
plt.show()

### Prime Video

In [ ]:
data_prime_top = data_prime[data_prime['IMDb']>8.5]
data_prime_top = data_prime_top[['Title', 'IMDb']].sort_values(ascending=False, by='IMDb')[:10]

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(x='IMDb',y='Title',data=data_prime_top, palette='husl')
plt.title('Top Prime Videos Movies')
plt.show()

## Average Runtime of Movies in each Platform

In [ ]:
dur_n = round(data_netflix['Runtime'].sum()/data_netflix.shape[0],2)
dur_h = round(data_hulu['Runtime'].sum()/data_hulu.shape[0],2)
dur_p = round(data_prime['Runtime'].sum()/data_prime.shape[0],2)
dur_d = round(data_disney['Runtime'].sum()/data_disney.shape[0],2)
print(dur_n, dur_h, dur_p, dur_d)

In [ ]:
duration_df = pd.DataFrame({
    'Streaming Platform' : ['NetFlix','Hulu','Prime Video','Disney+'],
    'Duration' : [dur_n, dur_h, dur_p, dur_d]
})
duration_df.head()

In [ ]:
duration_df.sort_values(ascending=False, by='Duration').iplot(kind="bar", x='Streaming Platform', 
                                                              xTitle='Streaming Platform', 
                                                              yTitle='Duration', title='Average Duration', color='Red')

### Thanks for visiting. Do upvote if you liked the kernel :-D